In [ ]:
import tables_io
import qp
import numpy as np

from rail.evaluation.dist_to_dist_evaluator import DistToDistEvaluator
from rail.evaluation.dist_to_point_evaluator import DistToPointEvaluator
from rail.evaluation.point_to_point_evaluator import PointToPointEvaluator
from rail.evaluation.single_evaluator import SingleEvaluator
from rail.core.stage import RailStage
from rail.core.data import QPHandle, TableHandle, QPOrTableHandle

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

# Load example Data

In [ ]:
import os
from rail.core.utils import find_rail_file
possible_local_file = './examples_data/evaluation_data/data/output_fzboost.hdf5'
if os.path.exists(possible_local_file):
    pdfs_file = os.path.abspath(possible_local_file)
else:
    pdfs_file = 'examples_data/evaluation_data/data/output_fzboost.hdf5'
    try:
        os.makedirs(os.path.dirname(pdfs_file))
    except FileExistsError:
        pass
    curl_com = f"curl -o {pdfs_file} https://portal.nersc.gov/cfs/lsst/PZ/output_fzboost.hdf5"
    os.system(curl_com)

ztrue_file = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')

In [ ]:
ensemble = DS.read_file(key='pdfs_data', handle_class=QPHandle, path=pdfs_file)
ztrue_data = DS.read_file('ztrue_data', TableHandle, ztrue_file)
#truth = DS.add_data('truth', ztrue_data()['photometry'], TableHandle, path=ztrue_file)
#truth_points = DS.add_data('truth_points', ztrue_data()['photometry']['redshift'], TableHandle, path=ztrue_file)

# Dist to Dist Evaluation

In [ ]:
# 'cvm' takes about 3.5 minutes to run
# 'ad' takes about ~4 minutes to run
# 'ks' takes about 2.75 minutes to run
# 'kld' takes about X minutes to run

stage_dict = dict(
    metrics=['cvm', 'ks', 'rmse', 'kld', 'ad'],
    _random_state=None,
)

dtd_stage = DistToDistEvaluator.make_stage(name='dist_to_dist', **stage_dict)
dtd_stage_single = DistToDistEvaluator.make_stage(name='dist_to_dist', force_exact=True, **stage_dict)

In [ ]:
# Parallelized implementation
dtd_results = dtd_stage.evaluate(ensemble, ensemble)

In [ ]:
# Non-parallelized, exact implementation
dtd_results_single = dtd_stage_single.evaluate(ensemble, ensemble)

In [ ]:
results_df = tables_io.convertObj(dtd_results(), tables_io.types.PD_DATAFRAME)
results_df_single = tables_io.convertObj(dtd_results_single(), tables_io.types.PD_DATAFRAME)
results_df

In [ ]:
results_df_single

# Dist to Point Evaluation

In [ ]:
stage_dict = dict(
    metrics=['cdeloss', 'pit', 'brier'],
    _random_state=None,
    metric_config={
        'brier': {'limits':(0,3.1)},
        'pit':{'tdigest_compression': 1000},
    }
)
dtp_stage = DistToPointEvaluator.make_stage(name='dist_to_point', **stage_dict)
dtp_stage_single = DistToPointEvaluator.make_stage(name='dist_to_point', force_exact=True, **stage_dict)

In [ ]:
dtp_results = dtp_stage.evaluate(ensemble, ztrue_data)
results_df = tables_io.convertObj(dtp_results['summary'](), tables_io.types.PD_DATAFRAME)

dtp_pit = dtp_stage.get_handle('single_distribution_summary').read()['pit']
results_df

In [ ]:
dtp_results_single = dtp_stage_single.evaluate(ensemble, ztrue_data)
results_df_single = tables_io.convertObj(dtp_results_single['summary'](), tables_io.types.PD_DATAFRAME)

dtp_pit_single = dtp_stage_single.get_handle('single_distribution_summary').read()['pit']
results_df_single

In [ ]:
import matplotlib.pyplot as plt

xgrid = np.linspace(0.05,0.95,100)
a_pdf = dtp_pit.pdf(xgrid)
b_pdf = dtp_pit_single.pdf(xgrid)

plt.figure()
plt.plot(xgrid, np.squeeze(a_pdf), label='parallelized, tdigest approximation')
plt.plot(xgrid, np.squeeze(b_pdf), label='non-parallelized, exact')
plt.legend()
plt.show()

# Point to Point Evaluation

In [ ]:
stage_dict = dict(
    metrics=['point_stats_ez', 'point_stats_iqr', 'point_bias', 'point_outlier_rate', 'point_stats_sigma_mad'],
    _random_state=None,
    hdf5_groupname='photometry',
    point_estimate_key='zmode',
    chunk_size=10000,
    metric_config={
        'point_stats_iqr':{'tdigest_compression': 100},
    }
)
ptp_stage = PointToPointEvaluator.make_stage(name='point_to_point', **stage_dict)
ptp_stage_single = PointToPointEvaluator.make_stage(name='point_to_point', force_exact=True, **stage_dict)

In [ ]:
ptp_results = ptp_stage.evaluate(ensemble, ztrue_data)
results_summary = tables_io.convertObj(ptp_stage.get_handle('summary')(), tables_io.types.PD_DATAFRAME)
results_summary

In [ ]:
ptp_results_single = ptp_stage_single.evaluate(ensemble, ztrue_data)
results_summary_single = tables_io.convertObj(ptp_stage_single.get_handle('summary')(), tables_io.types.PD_DATAFRAME)
results_summary_single

In [ ]:
truth = ztrue_data()['photometry']['redshift']
estimates = np.squeeze(ensemble().ancil['zmode'])
#truth_points = DS.add_data('truth_points', ztrue_data()['photometry']['redshift'], TableHandle, path=ztrue_file)

In [ ]:
check_iqr = qp.metrics.point_estimate_metric_classes.PointSigmaIQR().evaluate(estimates, truth)

In [ ]:
check_iqr

In [ ]:
vv = (estimates- truth)/(1.+truth)

In [ ]:
inputs = {
    'pdfs_data':'examples_data/evaluation_data/data/output_fzboost.hdf5',
    'ztrue_data':'examples_data/test_dc2_validation_9816.hdf5',
}
outputs = {
    'output':'output.hdf5',
    'summary':'summary.hdf5',
}

In [ ]:
from rail.core import RailPipeline

In [ ]:
pipe = RailPipeline()

In [ ]:
pipe.add_stage(ptp_stage)

In [ ]:
pipe.initialize(overall_inputs=inputs, run_config={'output_dir':'.', 'log_dir':'.', 'resume':False}, stages_config=None)

In [ ]:
pipe.save('eval_pipe.yaml')

## Single Evaluator

In [ ]:
stage_dict = dict(
    metrics=['cvm', 'ks', 'omega', 'kld', 'cdeloss', 'point_stats_ez', 'point_stats_iqr'],
    _random_state=None,
    hdf5_groupname='photometry',
    point_estimates=['zmode'],
    truth_point_estimates=['redshift'],
    chunk_size=1000,
)
ensemble_d = DS.add_data('pdfs_data_2', None, QPOrTableHandle, path=pdfs_file)
ztrue_data_d = DS.add_data('ztrue_data_2', None, QPOrTableHandle, path=ztrue_file)

single_stage = SingleEvaluator.make_stage(name='single', **stage_dict)
single_stage_single = SingleEvaluator.make_stage(name='single', force_exact=True, **stage_dict)

In [ ]:
single_results = single_stage.evaluate(ensemble_d, ztrue_data_d)

In [ ]:
single_results_single = single_stage_single.evaluate(ensemble_d, ztrue_data_d)

In [ ]:
single_stage.get_handle('output')()

In [ ]:
single_stage.get_handle('summary')()

In [ ]:
single_stage_single.get_handle('output')()

In [ ]:
single_stage_single.get_handle('summary')()